### Construct a vertical grid following Stewart et al. (2017)
This script is adapted from the [Stewart et al., 2017](https://github.com/kialstewart/vertical_grid_for_ocean_models/blob/master/build_vertical_grid_kds.py) script to construct a grid used with the MITgcm. The values of `drF` are to be copied to the `input/data` namelist. Additionally, a netcdf file is created for reference.

In [1]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

In [2]:
setup = "ESII_SO"
confi = "RCSI100.L55_ctlv30_030W"

Choose some parameters for the vertical grid

In [3]:
# Maximum depth (meters) of the grid
H = 4000.0

# Maximum grid spacing (meters) in the deep ocean
dzd = 150.0

# Minimum grid spacing (meters) at the surface
min_dz = 2.0

# Sharpness of the hyperbolic tangent (initial value should be adjusted)
depfac = 0.95

# Name of the file to be written that contains the grid
output_name = setup + "-" + confi + "-vertical_grid.nc"

### Build the grid
First, define the functional form of the vertical grid.

In [4]:
# the function is {tanh(pi*m/H)*dz_max + epsilon}, which is epsilon at the surface and dz_max at H
def f_all(kk, H, depfac, dzd, epsilon):
    return np.tanh(np.pi * ((kk) / (H * depfac))) * (dzd) + epsilon 

This is where the magic happens: an iterative process that takes a step from the current end (deepest point) of the grid along the function to find the next point. After that, we need to relocate the initial grid vertically so that the grid spacing at the surface is min_dz. All this is done in a function so we can interactively change the paraemters later.

In [6]:
def con_grid(H, min_dz, depfac, dzd):
    # this is a small number needed to begin the iteration
    epsilon = 0.001
    # Then, make the first two entries of the initial grid; these will be 0 and epsilon for both z and dz.
    delta_z = [0, epsilon * 1.0]
    prop_z = [0, epsilon * 1.0]
    while prop_z[-1] + delta_z[-1] < 1.2 * H:
        aa = np.linspace(1.0, 1.5, 10000)
        bb = np.zeros([len(aa)])
        loopkill = 1.0
        ii = 0
        while loopkill > 0:
            bb[ii] = (f_all(prop_z[-1] + (delta_z[-1] * aa[ii]), H, depfac, dzd, epsilon)) - (delta_z[-1] * aa[ii])
            loopkill = bb[ii]
            ii += 1
        aa_bb = np.polyfit(aa[:ii-1], bb[:ii-1], 1)
        dznew = (delta_z[-1] * (np.abs(aa_bb[1] / aa_bb[0])))
        delta_z = np.append(delta_z, dznew)
        prop_z = np.append(prop_z, (prop_z[-1] + delta_z[-1]))
    # find where the initial grid is min_dz (the surface resolution)
    new_surf = np.max(np.where(delta_z <= min_dz)) 
    # make a new grid that shifts the initial grid vertically
    real_prop_z = prop_z[new_surf:] - prop_z[new_surf] 
    # make a new dz for this new grid
    real_delta_z = delta_z[new_surf + 1:] 
    # cut the new grid off at desired depth, H
    real_prop_z = real_prop_z[np.where(real_prop_z < H)]
    # and the new dz too
    real_delta_z = real_delta_z[np.where(real_prop_z < H)]
    return real_prop_z, real_delta_z

In [7]:
final_prop_z, final_delta_z = con_grid(H, min_dz, depfac, dzd)

In [8]:
Z = -(final_prop_z + final_delta_z/2)
Zp1 = -np.append(final_prop_z[:], final_prop_z[-1] + final_delta_z[-1])
Zu = -np.append(final_prop_z[1::], final_prop_z[-1] + final_delta_z[-1])
Zl = -final_prop_z
drC = np.append(np.append(0 - Z[0], Z[0:-1] - Z[1::]), -Zu[-1] + Z[-1])
drF = final_delta_z

In [9]:
print(drF)

[  2.12950489   2.4308905    2.77490323   3.1675587    3.61571496
   4.1271872    4.71087665   5.37691441   6.13682095   7.00368101
   7.99233291   9.11956927  10.40434342  11.86797199  13.5343178
  15.42992871  17.58409559  20.02877577  22.79830608  25.92880133
  29.4571029   33.41911167  37.8473202   42.7673766   48.19359034
  54.12347726  60.5317637   67.36473409  74.53630909  81.92757038
  89.39130539  96.76222993 103.87196047 110.56607025 116.71949717
 122.24691006 127.10620628 131.29541339 134.84493639 137.80768112
 140.2493299  142.24020421 143.84931689 145.14057911 146.17078598
 146.98889983 147.63617797 148.14678645 148.54864422 148.86433209
 149.11196865 149.30600265 149.45790119 149.57673112 149.66964071]


In [10]:
len(drF)

55

In [11]:
ds_out = xr.Dataset(coords={"Z": (["Z"], Z),
                            "Zp1": (["Zp1"], Zp1),
                            "Zu": (["Zu"], Zu),
                            "Zl": (["Zl"], Zl),
                            "drC": (["Zp1"], drC),
                            "drF": (["Z"], drF),
                            })
ds_out["Z"].attrs = {"standard_name": "depth", 
                     "long_name": "vertical coordinate of cell center",
                     "units": "m",
                     "positive": "up",
                     "axis": "Z"}
ds_out["Zp1"].attrs = {"standard_name": "depth_at_w_location", 
                       "long_name": "vertical coordinate of cell interface",
                       "units": "m",
                       "positive": "up",
                       "axis": "Z",
                       "c_grid_axis_shift": (-0.5, 0.5)}
ds_out["Zu"].attrs = {"standard_name": "depth_at_lower_w_location", 
                      "long_name": "vertical coordinate of lower cell interface",
                      "units": "m",
                      "positive": "up",
                      "axis": "Z",
                      "c_grid_axis_shift": 0.5}
ds_out["Zl"].attrs = {"standard_name": "depth_at_upper_w_location", 
                      "long_name": "vertical coordinate of upper cell interface",
                      "units": "m",
                      "positive": "up",
                      "axis": "Z",
                      "c_grid_axis_shift": -0.5}
ds_out["drC"].attrs = {"standard_name": "cell_z_size_at_w_location", 
                       "long_name": "cell z size",
                       "units": "m"}
ds_out["drF"].attrs = {"standard_name": "cell_z_size", 
                       "long_name": "cell z size",
                       "units": "m"}
ds_out.attrs = {"input maximum depth H": str(H),
                "actual maximum depth": str(-np.min(Zp1)),
                "input minimum grid spacing min_dz": str(min_dz),
                "actual minimum grid spacing": str(np.min(drF)),
                "input maximum grid spazing dzd": str(dzd),
                "actual maximum grid spacing": str(np.max(drF)),
                "input factor depfac": str(depfac)}


In [12]:
ds_out.to_netcdf(output_name, engine='h5netcdf')

/home/jrieck/.venv/py39/lib/python3.9/site-packages/h5netcdf/core.py:481: H5pyDeprecationWarning: other_ds.dims.create_scale(ds, name) is deprecated. Use ds.make_scale(name) instead.
  h5ds.dims.create_scale(h5ds, scale_name)
